<a href="https://colab.research.google.com/github/milanchvojka-lgtm/starting-ragchatbot-codebase/blob/main/%E2%9C%85_Design_team_timesheet_analysis_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Instalace knihoven

In [ ]:
!pip install pandas openpyxl holidays plotly

# 2. Import knihoven

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import holidays
import plotly.graph_objects as go
import plotly.express as px

# 3. Načtení dat

In [ ]:
# Načtení dat z Excel souboru
df_analyza = pd.read_excel('Costlocker-Detailed-october.xlsx')  # Upravte název souboru podle vašeho skutečného souboru

# 4. Hlavní analyzátor

In [ ]:
class TimesheetAnalyzer:
    def __init__(self):
        self.df = None
        self.cz_holidays = holidays.CZ()
        self.categories = {
            'Jobs': ['jobs', 'job'],  # Opraveno - odstraněna dvojtečka
            'Reviews': ['review'],
            'Hiring': ['hiring']
        }

    def load_data(self, df):
        self.df = df
        self.df['Datum'] = pd.to_datetime(self.df['Datum'])

    def get_working_hours_for_month(self, year, month):
        first_day = datetime(year, month, 1)
        if month == 12:
            last_day = datetime(year + 1, 1, 1) - timedelta(days=1)
        else:
            last_day = datetime(year, month + 1, 1) - timedelta(days=1)

        working_days = 0
        current_day = first_day
        while current_day <= last_day:
            if current_day.weekday() < 5 and current_day not in self.cz_holidays:
                working_days += 1
            current_day += timedelta(days=1)
        return working_days * 8

    def calculate_fte(self, hours, date):
        year = date.year
        month = date.month
        monthly_hours = self.get_working_hours_for_month(year, month)
        return round(hours / monthly_hours, 2)

    def analyze_by_project(self):
        unique_months = pd.Period(self.df['Datum'].min(), freq='M')
        print("Přehled pracovních hodin v jednotlivých měsících:")
        working_hours = self.get_working_hours_for_month(unique_months.year, unique_months.month)
        print(f"- {unique_months}: {working_hours} hodin = 1 FTE")
        print("------------------------")

        project_analysis = (
            self.df
            .groupby(['Projekt'])
            .agg({
                'Natrackováno': ['sum', 'mean', 'count']
            })
            .round(2)
        )

        project_analysis.columns = ['Celkem hodin', 'Průměr hodin', 'Počet záznamů']

        project_analysis['FTE'] = project_analysis['Celkem hodin'].apply(
            lambda x: self.calculate_fte(x, unique_months.to_timestamp())
        )

        total_hours = project_analysis['Celkem hodin'].sum()
        project_analysis['Podíl (%)'] = (project_analysis['Celkem hodin'] / total_hours * 100).round(1)

        total_row = pd.DataFrame({
            'Celkem hodin': [total_hours],
            'Průměr hodin': [project_analysis['Průměr hodin'].mean()],
            'Počet záznamů': [project_analysis['Počet záznamů'].sum()],
            'FTE': [project_analysis['FTE'].sum()],
            'Podíl (%)': [100.0]
        }, index=['CELKEM'])

        project_analysis = pd.concat([project_analysis, total_row])
        return project_analysis

    def _get_ops_project(self):
        """Najde správný OPS projekt v datech"""
        ops_projects = [proj for proj in self.df['Projekt'].unique()
                       if 'ops' in proj.lower() and 'design' in proj.lower()]
        if not ops_projects:
            raise ValueError("Nenalezen žádný OPS projekt v datech")
        return ops_projects[0]

    def analyze_ops_activities(self):
        ops_project = self._get_ops_project()
        print(f"Analyzuji projekt: {ops_project}")

        ops_data = self.df[self.df['Projekt'] == ops_project].copy()

        def categorize_description(desc):
            if pd.isna(desc):
                return 'Ostatní'
            desc = str(desc).lower()
            for category, keywords in self.categories.items():
                if any(keyword in desc for keyword in keywords):
                    return category
            return 'Ostatní'

        ops_data['Kategorie'] = ops_data['Popis'].apply(categorize_description)

        category_analysis = ops_data.groupby('Kategorie').agg({
            'Natrackováno': 'sum'
        }).round(2)

        total_hours = category_analysis['Natrackováno'].sum()
        category_analysis['Podíl (%)'] = (category_analysis['Natrackováno'] / total_hours * 100).round(1)

        total_row = pd.DataFrame({
            'Natrackováno': [total_hours],
            'Podíl (%)': [100.0]
        }, index=['CELKEM'])

        category_analysis = pd.concat([category_analysis, total_row])
        return category_analysis

    def analyze_ops_by_person(self):
        ops_project = self._get_ops_project()
        ops_data = self.df[self.df['Projekt'] == ops_project].copy()

        def categorize_description(desc):
            if pd.isna(desc):
                return 'Ostatní'
            desc = str(desc).lower()
            for category, keywords in self.categories.items():
                if any(keyword in desc for keyword in keywords):
                    return category
            return 'Ostatní'

        ops_data['Kategorie'] = ops_data['Popis'].apply(categorize_description)

        person_category_analysis = ops_data.pivot_table(
            values='Natrackováno',
            index='Osoba',
            columns='Kategorie',
            aggfunc='sum',
            fill_value=0
        ).round(2)

        person_category_analysis['Celkem'] = person_category_analysis.sum(axis=1)
        total_hours = person_category_analysis['Celkem'].sum()
        person_category_analysis['Podíl (%)'] = (person_category_analysis['Celkem'] / total_hours * 100).round(1)

        return person_category_analysis

# 5. Příprava vizualizací

In [ ]:
def create_visualizations(analyzer):
    # Definice základních parametrů
    main_color = '#FF7CAC'
    light_color = '#FFD9E5'
    ops_activities_order = ['Jobs', 'Reviews', 'Hiring', 'Ostatní']

    # Společné nastavení pro všechny grafy
    layout_settings = {
        'margin': dict(l=250, r=500),
        'showlegend': True,
        'plot_bgcolor': 'white',
        'paper_bgcolor': 'white',
        'hoverlabel': dict(bgcolor='white'),
        'width': 1200,
    }

    # ---------- ČÁST 1: Generování datových přehledů ----------
    print("Analýza podle projektů:")
    project_data = analyzer.analyze_by_project()
    print(project_data)

    person_fte = analyzer.df.groupby('Osoba')['Natrackováno'].sum().apply(
        lambda x: analyzer.calculate_fte(x, pd.Period(analyzer.df['Datum'].min(), freq='M').to_timestamp())
    ).sort_values()

    print("\nFTE podle osob:")
    total_fte = 0
    for person, fte in person_fte.items():
        print(f"- {person}: {fte:.2f} FTE")
        total_fte += fte
    print(f"Celkem FTE všech osob: {total_fte:.2f}")
    print("------------------------")

    print("\nAnalýza OPS aktivit:")
    ops_activities = analyzer.analyze_ops_activities()
    print(ops_activities)

    print("\nAnalýza OPS aktivit podle osob:")
    ops_by_person = analyzer.analyze_ops_by_person()
    print(ops_by_person)

    # ---------- ČÁST 2: Export dat do Excelu ----------
    with pd.ExcelWriter('timesheet_analysis.xlsx') as writer:
        project_data.to_excel(writer, sheet_name='Projekty')

        person_fte_df = pd.DataFrame({
            'Osoba': person_fte.index,
            'FTE': person_fte.values
        })
        person_fte_df.loc[len(person_fte_df)] = ['CELKEM', person_fte.sum()]
        person_fte_df.to_excel(writer, sheet_name='FTE podle osob', index=False)

        ops_activities.to_excel(writer, sheet_name='OPS aktivity')
        ops_by_person.to_excel(writer, sheet_name='OPS aktivity podle osob')

    print("\nData byla exportována do souboru 'timesheet_analysis.xlsx'")
    print("\nVizualizace dat:")
    print("------------------------")

    # ---------- ČÁST 3: Interaktivní vizualizace ----------
    # 1. Graf FTE a podílu času podle projektů
    project_data_sorted = project_data[:-1].sort_values('FTE')
    max_fte = project_data_sorted['FTE'].max()
    fig1 = go.Figure(data=[go.Bar(
        x=project_data_sorted['FTE'],
        y=project_data_sorted.index,
        orientation='h',
        marker_color=main_color,
        text=[f"{fte:.2f} FTE ({pct:.1f}%)" for fte, pct in
              zip(project_data_sorted['FTE'], project_data_sorted['Podíl (%)'])],
        textposition='outside'
    )])
    fig1.update_layout(
        title='FTE a podíl času podle projektů',
        xaxis_title='FTE',
        xaxis=dict(
            range=[0, max_fte * 1.4],
            showgrid=True,
            gridcolor='lightgrey',
            gridwidth=1,
            zeroline=True,
            zerolinecolor='lightgrey',
            zerolinewidth=1
        ),
        yaxis=dict(
            showgrid=True,
            gridcolor='lightgrey',
            gridwidth=1
        ),
        **layout_settings
    )
    fig1.show()

    # X. Graf s přehledem projektů
    projects_sorted = project_data[:-1].sort_values('Celkem hodin', ascending=True)
    max_hours = projects_sorted['Celkem hodin'].max()
    fig_projects = go.Figure(data=[go.Bar(
        x=projects_sorted['Celkem hodin'],
        y=projects_sorted.index,
        orientation='h',
        marker_color=main_color,
        text=[f"{hours:.1f}h ({pct:.1f}%)" for hours, pct in
              zip(projects_sorted['Celkem hodin'], projects_sorted['Podíl (%)'])],
        textposition='outside'
    )])
    fig_projects.update_layout(
        title='Rozdělení práce mezi projekty',
        xaxis_title='Počet hodin',
        xaxis=dict(
            range=[0, max_hours * 1.4],
            showgrid=True,
            gridcolor='lightgrey',
            gridwidth=1,
            zeroline=True,
            zerolinecolor='lightgrey',
            zerolinewidth=1
        ),
        yaxis=dict(
            showgrid=True,
            gridcolor='lightgrey',
            gridwidth=1
        ),
        **layout_settings
    )
    fig_projects.show()

    # 2. Graf FTE podle osob
    max_person_fte = person_fte.max()
    fig2 = go.Figure(data=[go.Bar(
        x=person_fte.values,
        y=person_fte.index,
        orientation='h',
        marker_color=main_color,
        text=[f"{x:.2f} FTE" for x in person_fte.values],
        textposition='outside'
    )])
    fig2.update_layout(
        title='FTE podle osob',
        xaxis_title='FTE',
        xaxis=dict(
            range=[0, max_person_fte * 1.4],
            showgrid=True,
            gridcolor='lightgrey',
            gridwidth=1,
            zeroline=True,
            zerolinecolor='lightgrey',
            zerolinewidth=1
        ),
        yaxis=dict(
            showgrid=True,
            gridcolor='lightgrey',
            gridwidth=1
        ),
        **layout_settings
    )
    fig2.show()

    # 3. Srovnávací graf plánovaného a skutečného FTE
    planned_fte = {
        'Milan Chvojka': 0.9,
        'Jiří Martínek': 0.5,
        'Petra Panáková': 0.4,
        'Tomáš Brza': 1.0 # Updated Tomáš Brza's FTE to 1.0
    }

    # Filter planned_fte to include only people present in actual data
    planned_fte_filtered = {person: fte for person, fte in planned_fte.items() if person in person_fte.index}
    actual_fte = person_fte[list(planned_fte_filtered.keys())]

    max_comparison = max(max(planned_fte_filtered.values()), actual_fte.max())

    fig_comparison = go.Figure(data=[
        go.Bar(
            name='Plánovaný FTE',
            x=list(planned_fte_filtered.values()),
            y=list(planned_fte_filtered.keys()),
            orientation='h',
            marker_color=light_color,
            text=[f"{x:.2f} FTE" for x in planned_fte_filtered.values()],
            textposition='outside'
        ),
        go.Bar(
            name='Skutečný FTE',
            x=actual_fte.values,
            y=actual_fte.index,
            orientation='h',
            marker_color=main_color,
            text=[f"{x:.2f} FTE" for x in actual_fte.values],
            textposition='outside'
        )
    ])
    fig_comparison.update_layout(
        title='Porovnání plánovaného a skutečného FTE',
        xaxis_title='FTE',
        xaxis=dict(
            range=[0, max_comparison * 1.4],
            showgrid=True,
            gridcolor='lightgrey',
            gridwidth=1,
            zeroline=True,
            zerolinecolor='lightgrey',
            zerolinewidth=1
        ),
        yaxis=dict(
            showgrid=True,
            gridcolor='lightgrey',
            gridwidth=1
        ),
        barmode='group',
        **layout_settings
    )
    fig_comparison.show()

    # 4. Graf rozdělení OPS aktivit
    ops_data = analyzer.analyze_ops_activities()
    ops_data_reordered = ops_data.reindex(ops_activities_order)
    max_ops = ops_data_reordered['Natrackováno'].max()

    fig3 = go.Figure(data=[go.Bar(
        x=ops_data_reordered['Natrackováno'][:-1],
        y=ops_data_reordered.index[:-1],
        orientation='h',
        marker_color=main_color,
        text=[f"{ops_data_reordered['Natrackováno'][i]:.2f}h ({ops_data_reordered['Podíl (%)'][i]:.1f}%)"
              for i in ops_data_reordered.index[:-1]],
        textposition='outside'
    )])
    fig3.update_layout(
        title='Rozdělení OPS aktivit',
        xaxis_title='Hodiny',
        xaxis=dict(
            range=[0, max_ops * 1.4],
            showgrid=True,
            gridcolor='lightgrey',
            gridwidth=1,
            zeroline=True,
            zerolinecolor='lightgrey',
            zerolinewidth=1
        ),
        yaxis=dict(
            showgrid=True,
            gridcolor='lightgrey',
            gridwidth=1
        ),
        **layout_settings
    )
    fig3.show()

    # 5. Individuální grafy OPS aktivit pro každou osobu
    for person in ops_by_person.index:
        person_data = pd.Series(0.0, index=ops_activities_order)
        for activity in ops_activities_order:
            if activity in ops_by_person.columns:
                person_data[activity] = ops_by_person.loc[person, activity]

        max_person_ops = person_data.max()
        fig = go.Figure(data=[go.Bar(
            x=person_data.values,
            y=person_data.index,
            orientation='h',
            marker_color=main_color,
            text=[f"{val:.2f}h" for val in person_data.values],
            textposition='outside'
        )])
        fig.update_layout(
            title=f'OPS aktivity - {person}',
            xaxis_title='Hodiny',
            xaxis=dict(
                range=[0, max_person_ops * 1.4],
                showgrid=True,
                gridcolor='lightgrey',
                gridwidth=1,
                zeroline=True,
                zerolinecolor='lightgrey',
                zerolinewidth=1
            ),
            yaxis=dict(
                showgrid=True,
                gridcolor='lightgrey',
                gridwidth=1
            ),
            **layout_settings
        )
        fig.show()

# 6. (volitelně) Analýza popisů OPS aktivit

In [ ]:
# Nejprve vytvoříme analyzátor
analyzer = TimesheetAnalyzer()
analyzer.load_data(df_analyza)

# Pak můžeme spustit analýzu popisů
def analyze_ops_descriptions(analyzer):
    # Získání pouze OPS dat
    ops_data = analyzer.df[analyzer.df['Projekt'] == 'Design tým OPS_2025'].copy()

    # Definice kategorií a klíčových slov
    categories = {
        'Jobs': ['jobs', 'job'],
        'Reviews': ['review'],
        'Hiring': ['hiring']
    }

    def find_category_and_keywords(desc):
        desc = str(desc).lower()
        for category, keywords in categories.items():
            if any(keyword in desc for keyword in keywords):
                matching_keywords = [kw for kw in keywords if kw in desc]
                return category, ', '.join(matching_keywords)
        return 'Ostatní', ''

    ops_data['Kategorie'], ops_data['Nalezená klíčová slova'] = zip(*ops_data['Popis'].apply(find_category_and_keywords))
    ops_data = ops_data.sort_values('Datum')

    print(f"Celkový počet OPS záznamů: {len(ops_data)}")
    print("\nRozdělení podle kategorií:")
    print(ops_data['Kategorie'].value_counts())

    print("\nDetail záznamů:")
    for category in ['Ostatní'] + list(categories.keys()):
        category_data = ops_data[ops_data['Kategorie'] == category]
        if len(category_data) > 0:
            print(f"\n{category} ({len(category_data)} záznamů):")
            for _, row in category_data.iterrows():
                print(f"- {row['Datum'].strftime('%Y-%m-%d')} | {row['Osoba']} | {row['Natrackováno']}h | {row['Popis']}")
                if category == 'Ostatní':
                    print(f"  (Tento záznam nebyl přiřazen k žádné kategorii)")

# Spuštění analýzy
analyze_ops_descriptions(analyzer)

Celkový počet OPS záznamů: 180

Rozdělení podle kategorií:
Kategorie
Jobs       136
Hiring      31
Reviews     13
Name: count, dtype: int64

Detail záznamů:

Jobs (136 záznamů):
- 2025-10-01 | Petra Panáková | 1.0h | Jobs: Škola improvizace odpoved na email, plan aktivit
- 2025-10-01 | Tomáš Brza | 0.75h | Job: Eurowag
- 2025-10-01 | Jiří Martínek | 1.25h | Jobs: Stada ft Milan
- 2025-10-01 | Tomáš Brza | 0.5h | Job: Eurowag
- 2025-10-01 | Petra Panáková | 0.5h | Jobs: MSD příprava Vládi na chemistry call
- 2025-10-01 | Milan Chvojka | 0.5h | Jobs: MSD rehearsal Vládi
- 2025-10-01 | Petra Panáková | 1.0h | Jobs: MSD příprava Martiny na chemistry call + příprava miro boardu
- 2025-10-01 | Milan Chvojka | 1.0h | Jobs: Sales status
- 2025-10-01 | Tomáš Brza | 1.0h | Jobs: Relatoo a iné
- 2025-10-01 | Petra Panáková | 0.5h | Jobs: MSD příprava na chemistry call příklady s Milanem
- 2025-10-01 | Petra Panáková | 0.5h | Jobs: MSD rehearsal Martiny
- 2025-10-01 | Petra Panáková | 1.0h | Jobs:

# 7. Export dat a tvorba vizualizací

In [ ]:
# Vytvoření instance analyzátoru a načtení dat
analyzer = TimesheetAnalyzer()
analyzer.load_data(df_analyza)

# Spuštění vizualizací
create_visualizations(analyzer)

Analýza podle projektů:
Přehled pracovních hodin v jednotlivých měsících:
- 2025-10: 176 hodin = 1 FTE
------------------------
                         Celkem hodin  Průměr hodin  Počet záznamů   FTE  \
Design tým Interní_2025        108.00      0.800000            135  0.61   
Design tým OPS_2025            198.75      1.100000            180  1.13   
Design tým PR_2025              40.50      1.560000             26  0.23   
Design tým R&D_2025             48.75      1.110000             44  0.28   
Guiding_2025                    29.00      0.830000             35  0.16   
UX Maturity 🙌                    1.00      0.500000              2  0.01   
CELKEM                         426.00      0.983333            422  2.42   

                         Podíl (%)  
Design tým Interní_2025       25.4  
Design tým OPS_2025           46.7  
Design tým PR_2025             9.5  
Design tým R&D_2025           11.4  
Guiding_2025                   6.8  
UX Maturity 🙌                  0.2  
CELK

Analyzuji projekt: Design tým OPS_2025
